<a href="https://colab.research.google.com/github/ManuelaCastano/CuadernosyEjercicios/blob/main/Rectificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUNTO 4



In [ ]:
!pip install numpy matplotlib streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 752.5 kB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
%%writefile rectificador.py

import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from scipy.fft import fft, fftfreq
from scipy.integrate import simps

def generate_waveform(frequency, amplitude, sample_rate, duration):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    waveform = amplitude * np.sin(2 * np.pi * frequency * t)
    return t, waveform

def rectified_waveform(signal):
    return np.abs(signal)

def calculate_thd(signal, sample_rate):
    n = len(signal)
    yf = fft(signal)
    xf = fftfreq(n, 1 / sample_rate)

    # Eliminar las frecuencias negativas
    xf = xf[:n // 2]
    yf = 2.0 / n * np.abs(yf[:n // 2])

    # THD
    fundamental_freq = xf[np.argmax(yf[1:]) + 1]  # Primer pico significativo
    fundamental_amplitude = yf[np.argmax(yf[1:]) + 1]
    harmonics = yf[2:]  # Omitir el fundamental para los armónicos
    thd = np.sqrt(np.sum(harmonics**2)) / fundamental_amplitude

    return thd

def calculate_power_factor(signal, sample_rate, R, C=None):
    n = len(signal)
    t = np.arange(n) / sample_rate
    average_power = np.mean(signal**2)

    if C is not None:
        omega = 2 * np.pi * 50  # Frecuencia de red (por ejemplo, 50 Hz)
        impedance = R + 1j * (1 / (omega * C))  # Impedancia total
    else:
        impedance = R

    current_rms = np.sqrt(np.mean(signal**2))
    voltage_rms = np.sqrt(average_power) / R
    power_factor = voltage_rms / (current_rms * abs(impedance))

    return power_factor

def main():
    st.title('Análisis de Rectificador de Onda Completa')

    frequency = st.slider('Frecuencia (Hz)', 50, 1000, 50)
    amplitude = st.slider('Amplitud (V)', 1, 10, 1)
    sample_rate = st.slider('Tasa de Muestreo (Hz)', 1000, 100000, 1000)
    duration = st.slider('Duración (s)', 0.1, 5.0, 1.0)
    R = st.slider('Resistencia (Ohm)', 1, 100, 10)
    C = st.number_input('Capacitor (Faradios)', 0.0, 0.01, 0.0)

    t, waveform = generate_waveform(frequency, amplitude, sample_rate, duration)
    rectified_signal = rectified_waveform(waveform)

    thd = calculate_thd(rectified_signal, sample_rate)
    power_factor = calculate_power_factor(rectified_signal, sample_rate, R, C if C > 0 else None)

    st.write(f'THD (Distorsión Armónica Total): {thd:.2f}')
    st.write(f'Factor de Potencia: {power_factor:.2f}')

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(t, waveform)
    plt.title('Señal Original')
    plt.subplot(1, 2, 2)
    plt.plot(t, rectified_signal)
    plt.title('Señal Rectificada')
    st.pyplot(plt)

if __name__ == "__main__":
    main()


Writing rectificador.py


In [ ]:
token = '2lLjHoDPtNqGFhA3XCMXaV7TSzC_2WMetgXt54NgDAv4qBuZE' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run rectificador.py --server.port 9600 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='9600', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://c7f8-34-74-177-65.ngrok-free.app


In [ ]:
#exit("Stopping the execution")